In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from geopy.geocoders import Nominatim
import json
import requests
from pandas.io.json import json_normalize

# import kmeans from clustering stage
from sklearn.cluster import KMeans
import folium

In [3]:
rent_df = pd.read_csv("toronto_rent_data.csv")
geo_coords = pd.read_csv("Geospatial_Coordinates.csv")

In [4]:
# make a copy of the df for testing codes
scraped_df = rent_df

In [5]:
rent_df.head()

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
0,"$3,300",85 Wood Street,M4Y,3,2,810
1,"$2,295",300 Balliol Street,M4S,1,1,600
2,"$2,300",8 Fieldway Road,M8Z,1,1,NaN
3,"$1,950",160 Flemington Road,M6A,2,2,630
4,"$2,045",117 Gerrard Street East,M5B,1,1,NaN


In [6]:
geo_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# check for null values 
rent_df.isnull().sum()
# missing 5 zip codes and 945 sqft of units

Rent               0
Address            0
Zip                5
num_bedrooms       0
num_bathrooms      0
num_sqft         945
dtype: int64

In [8]:
# check data type for each features
rent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rent           2450 non-null   object
 1   Address        2450 non-null   object
 2   Zip            2445 non-null   object
 3   num_bedrooms   2450 non-null   object
 4   num_bathrooms  2450 non-null   object
 5   num_sqft       1505 non-null   object
dtypes: object(6)
memory usage: 115.0+ KB


### Convert Datatypes (Data Cleaning)

In [9]:
scraped_df['Rent'][0].replace("$","").replace(",","")
# scraped_df['Rent'][0].replace(",","")

'3300'

In [10]:
# delete the dollar sign and comma before turning to float
for i in range(len(rent_df['Rent'])):
    rent_df["Rent"][i] = rent_df["Rent"][i].replace("$", "").replace(",", "")


In [11]:
rent_df.head()

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
0,3300,85 Wood Street,M4Y,3,2,810
1,2295,300 Balliol Street,M4S,1,1,600
2,2300,8 Fieldway Road,M8Z,1,1,NaN
3,1950,160 Flemington Road,M6A,2,2,630
4,2045,117 Gerrard Street East,M5B,1,1,NaN


In [12]:
rent_df.iloc[0,:]

Rent                       3300
Address          85 Wood Street
Zip                         M4Y
num_bedrooms                  3
num_bathrooms                 2
num_sqft                    810
Name: 0, dtype: object

In [13]:
# create empty dataframe to store ranged data
empty_ranged_df = pd.DataFrame(columns = rent_df.columns)

# append all values in the dataset that have the "-" in cells
for features in rent_df.columns:
#     print(features)
    empty_ranged_df = pd.concat([empty_ranged_df, (rent_df[rent_df[features].astype(str).str.contains("-")])])

In [14]:
# dislay all data
# pd.set_option("display.max_rows",None)

In [15]:
empty_ranged_df

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
191,1750 - 2000,7895 Jane Street,L4K,1 - 2,1 - 2,500-800
438,2200 - 2300,77 Shuter Street,M5B,1,1 ½,550-650
476,1750 - 2050,38 Cameron Street,M5T,1,1 - 1 ½,521-654
480,1895 - 3875,1 Edgewater Drive,M5A,1 - 2,1 - 2 ½,466-1119
563,1950 - 2850,16 Yonge Street,M5E,1 - 2,1 - 2,478-815
...,...,...,...,...,...,...
1535,1680 - 2000,1030 King Street West,M6K,0 - 1,1,370-450
1704,1700 - 4200,50 John Street,M5V,0 - 2,1 - 2,365-1230
2167,1850 - 2250,60 Ann OReilly Road,M2J,1 - 2,1 - 2,580-828
2372,1850 - 2400,7 Mabelle Avenue,M9A,1 - 2,1 - 2,488-779


In [16]:
empty_ranged_df.shape

(106, 6)

In [17]:
empty_ranged_df.index

Int64Index([ 191,  438,  476,  480,  563,  566,  589,  608,  732,  773,
            ...
            1105, 1138, 1196, 1418, 1480, 1535, 1704, 2167, 2372, 2398],
           dtype='int64', length=106)

In [18]:
# delete duplicate indexes
empty_ranged_df.index.drop_duplicates(keep='first')

Int64Index([ 191,  438,  476,  480,  563,  566,  589,  608,  732,  773,  825,
             830,  836,  890,  962, 1041, 1043, 1071, 1090, 1105, 1138, 1196,
            1418, 1480, 1535, 1635, 1642, 1704, 2167, 2321, 2365, 2372, 2398,
             186, 1045,  692],
           dtype='int64')

In [19]:
empty_ranged_df = empty_ranged_df.loc[~empty_ranged_df.index.duplicated(keep="first")]

In [20]:
# check if changes are made
empty_ranged_df.shape

(36, 6)

In [21]:
empty_ranged_df.head()

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
191,1750 - 2000,7895 Jane Street,L4K,1 - 2,1 - 2,500-800
438,2200 - 2300,77 Shuter Street,M5B,1,1 ½,550-650
476,1750 - 2050,38 Cameron Street,M5T,1,1 - 1 ½,521-654
480,1895 - 3875,1 Edgewater Drive,M5A,1 - 2,1 - 2 ½,466-1119
563,1950 - 2850,16 Yonge Street,M5E,1 - 2,1 - 2,478-815


In [22]:
# select only data that don't contain range of values
rent_norange_df = rent_df[rent_df.index.isin(empty_ranged_df.index) == False]

In [23]:
# rent_norange_df should have 2414 rows because 2450 - 36 = 2414
rent_norange_df.shape

(2414, 6)

In [24]:
rent_df.shape

(2450, 6)

In [25]:
rent_norange_df['num_bedrooms'].unique()
rent_norange_df['num_bathrooms'].unique()


array(['2', '1', '1 ½', '3', '0 ½', '0', '2 ½'], dtype=object)

In [26]:
rent_norange_df['num_bathrooms'][0].replace("½",".5").replace(" ","")

'2'

In [27]:
rent_norange_df.head(3)

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
0,3300,85 Wood Street,M4Y,3,2,810
1,2295,300 Balliol Street,M4S,1,1,600
2,2300,8 Fieldway Road,M8Z,1,1,NaN


In [28]:
# reset index before looping through the changes
rent_norange_df = rent_norange_df.reset_index(drop=True)

In [29]:
rent_norange_df.loc[[185,186,187,188,189,190]]

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
185,1900,2916 Highway 7,L4K,1,0 ½,550
186,2300,2908 Highway 7,L4K,2,2,800899
187,2000,2916 Highway 7,L4K,1,1 ½,630
188,2000,7895 Jane Street,L4K,1,2,611
189,2200,7895 Jane Street,L4K,2,2,750
190,2250,7895 Jane Street,L4K,2,2,649


In [30]:
# change "1/2" into 0.5
for bath in range(rent_norange_df.shape[0]):
    rent_norange_df['num_bathrooms'][bath] = rent_norange_df['num_bathrooms'][bath].replace("½",".5").replace(" ","")

In [31]:
# check if changes are made
rent_norange_df['num_bathrooms'].unique()

array(['2', '1', '1.5', '3', '0.5', '0', '2.5'], dtype=object)

In [32]:
rent_norange_df.dtypes

Rent             object
Address          object
Zip              object
num_bedrooms     object
num_bathrooms    object
num_sqft         object
dtype: object

In [33]:
# convert datatypes
rent_norange_df["Rent"] = rent_norange_df["Rent"].astype(float)
rent_norange_df["num_bedrooms"] = rent_norange_df["num_bedrooms"].astype(float)
rent_norange_df["num_bathrooms"] = rent_norange_df["num_bathrooms"].astype(float)
rent_norange_df["num_sqft"] = rent_norange_df["num_sqft"].astype(float)


In [34]:
rent_norange_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2414 entries, 0 to 2413
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rent           2414 non-null   float64
 1   Address        2414 non-null   object 
 2   Zip            2410 non-null   object 
 3   num_bedrooms   2414 non-null   float64
 4   num_bathrooms  2414 non-null   float64
 5   num_sqft       1472 non-null   float64
dtypes: float64(4), object(2)
memory usage: 113.3+ KB


In [35]:
# match the postal codes
# check the number of postal codes that match
rent_df.loc[rent_df['Zip'].isin(geo_coords['Postal Code']), :].shape

(2300, 6)

In [36]:
# check for null values
rent_norange_df.isnull().sum()

Rent               0
Address            0
Zip                4
num_bedrooms       0
num_bathrooms      0
num_sqft         942
dtype: int64

Just going to drop all the null values because 942 null values is too much for this dataset (represents 1/3 of the data for num_sqft) and there's not point in keeping data that doesn't have a zip code because the whole point of this analysis is plot all the data on a map.

In [37]:
rent_norange_df.shape

(2414, 6)

In [38]:
# find the rows that contain null values for "Zip"
rent_norange_df.loc[rent_norange_df['Zip'].isnull()]

,Rent,Address,Zip,num_bedrooms,num_bathrooms,num_sqft
13,2080.0,35 Balmuto Street,NaN,1.0,1.0,610.0
93,1900.0,3560 St Clair Ave E | Unit: 418,NaN,1.0,1.0,700.0
1990,2500.0,50 Ann O'Reilly Road | Unit: 2,NaN,2.0,2.0,1075.0
2014,2950.0,85 The Donway W,NaN,2.0,2.0,1450.0


In [39]:
# remove null values
dropped_nulls_df = rent_norange_df.dropna()
one_bedroom_df = rent_norange_df.loc[rent_norange_df['num_bedrooms'] == 1].dropna()

In [40]:
dropped_nulls_df.shape
one_bedroom_df.shape

(934, 6)

In [41]:
dropped_nulls_df.loc[dropped_nulls_df['Zip'].isin(geo_coords["Postal Code"])].shape

(1342, 6)

In [42]:
dropped_nulls_df = dropped_nulls_df.reset_index(drop=True)
one_bedroom_df = one_bedroom_df.reset_index(drop=True)


#### Store Data into a database

In [43]:
from sqlalchemy import create_engine

In [44]:
connection_string = "postgres:postgres@localhost:5432/toronto-rental-properties"
engine = create_engine(f'postgresql://{connection_string}')

In [45]:
connection = engine.connect()

In [46]:
dropped_nulls_df.to_sql(name="all_data", con = engine, if_exists="replace",index = False)

In [47]:
# change column names for geo_coords before storing in pgadmin
geo_coords.rename(columns = ({"Postal Code": "postal_code"}), inplace = True)

In [48]:
geo_coords_columnNames = ['postal_code', "latitude", "longitude"]

In [49]:
geo_coords.columns = geo_coords_columnNames

In [50]:
one_bedroom_columnNames = ['rent', 'address', 'zip', 'num_bedrooms', 'num_bathrooms', 'num_sqft']

In [51]:
one_bedroom_df.columns = one_bedroom_columnNames

In [52]:
one_bedroom_df.head(4)

,rent,address,zip,num_bedrooms,num_bathrooms,num_sqft
0,2295.0,300 Balliol Street,M4S,1.0,1.0,600.0
1,2050.0,83 Redpath Avenue,M4S,1.0,1.0,562.0
2,2350.0,633 Bay Street,M5G,1.0,1.5,833.0
3,2250.0,88 Harbour Street,M5J,1.0,1.0,581.0


In [53]:
one_bedroom_df.to_sql(name="one_bedroom_data", con = engine, if_exists = "replace", index=False)


In [54]:
geo_coords.to_sql(name="geo_data", con = engine, if_exists = "replace", index = False)

In [74]:
pd.read_sql_query("SELECT * \
                    FROM one_bedroom_data \
                    LIMIT 5", connection)

,rent,address,zip,num_bedrooms,num_bathrooms,num_sqft
0,2295.0,300 Balliol Street,M4S,1.0,1.0,600.0
1,2050.0,83 Redpath Avenue,M4S,1.0,1.0,562.0
2,2350.0,633 Bay Street,M5G,1.0,1.5,833.0
3,2250.0,88 Harbour Street,M5J,1.0,1.0,581.0
4,1950.0,66 Portland Street,M5V,1.0,1.0,570.0


In [79]:
# join datasets with sql to flex my sql skills
sql_merged = pd.read_sql_query("SELECT g.postal_code,\
                               g.latitude,\
                               g.longitude,\
                               ob.rent,\
                               ob.address,\
                               ob.num_bedrooms,\
                               ob.num_bathrooms,\
                               ob.num_sqft\
                               FROM geo_data g\
                               JOIN one_bedroom_data ob\
                               ON ob.zip = g.postal_code", connection)

In [80]:
sql_merged

,postal_code,latitude,longitude,rent,address,num_bedrooms,num_bathrooms,num_sqft
0,M1H,43.773136,-79.239476,1925.0,36 Lee Centre Drive,1.0,1.0,750.0
1,M1H,43.773136,-79.239476,1950.0,38 Lee Centre Drive,1.0,1.0,650.0
2,M1H,43.773136,-79.239476,1950.0,11 Lee Centre Drive,1.0,1.0,650.0
3,M1H,43.773136,-79.239476,1850.0,8 Lee Centre Drive,1.0,1.0,650.0
4,M1H,43.773136,-79.239476,2000.0,88 Grangeway Avenue,1.0,1.0,650.0
...,...,...,...,...,...,...,...,...
856,M9C,43.643515,-79.577201,2000.0,2 Eva Road,1.0,1.0,650.0
857,M9C,43.643515,-79.577201,2100.0,235 Sherway Gardens Road,1.0,1.0,650.0
858,M9C,43.643515,-79.577201,2000.0,6 Eva Road,1.0,1.0,688.0
859,M9C,43.643515,-79.577201,2280.0,215 Sherway Gardens Road,1.0,2.0,743.0


In [83]:
# the rows should match sql_merged
one_bedroom_df.loc[one_bedroom_df['zip'].isin(geo_coords['postal_code'])].shape

(861, 6)

In [91]:
# group datasets with sql to flex my sql skills
sql_group_postal_code = pd.read_sql_query("SELECT g.postal_code,\
                                                  g.latitude,\
                                                  g.longitude,\
                                                  AVG(ob.rent) as average_rent,\
                                                  AVG(ob.num_bathrooms) as average_bathrooms,\
                                                  AVG(ob.num_sqft) as average_sqft \
                                            FROM geo_data g \
                                              JOIN one_bedroom_data ob \
                                                  ON ob.zip = g.postal_code \
                                            GROUP BY g.postal_code, \
                                                     g.latitude, \
                                                     g.longitude \
                                            ORDER BY g.postal_code;", connection)

In [92]:
sql_group_postal_code.head(4)

,postal_code,latitude,longitude,average_rent,average_bathrooms,average_sqft
0,M1H,43.773136,-79.239476,1902.272727,1.000000,618.727273
1,M1N,43.692657,-79.264848,2050.000000,1.000000,735.000000
2,M1P,43.757410,-79.273304,1936.580645,0.983871,624.129032
3,M1S,43.794200,-79.262029,1852.500000,1.100000,592.000000


In [94]:
# graph the coordinates
map_toronto = folium.Map(location = [sql_group_postal_code['latitude'][0], \
                                    sql_group_postal_code['longitude'][0]], zoom_start=10)

# add markers to map
for lat,lng,rent,bathrooms,sqft in zip(sql_group_postal_code['latitude'], sql_group_postal_code['longitude'],\
                                       sql_group_postal_code['average_rent'],sql_group_postal_code['average_bathrooms'],\
                                       sql_group_postal_code['average_sqft']):
    label = f"Price: {rent} \n Bathrooms: {bath} \n sqft: {sqft}"
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color = "#3186cc",
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

In [95]:
map_toronto

In [65]:
zip_group = one_bedroom_df.groupby(['zip']).mean()

In [66]:
zip_group.head()

,rent,num_bedrooms,num_bathrooms,num_sqft
zip,,,,
L3P,1675.000000,1.0,1.000000,549.000000
L3R,2000.000000,1.0,1.000000,750.000000
L3T,1990.000000,1.0,1.000000,600.000000
L4B,1830.000000,1.0,1.000000,580.000000
L4C,1897.111111,1.0,1.111111,758.555556


In [67]:
zip_group.shape

(68, 4)

In [68]:
zip_group = zip_group.reset_index().rename(columns={"zip": "postal_code"})

In [69]:
zip_group.head(3)

,postal_code,rent,num_bedrooms,num_bathrooms,num_sqft
0,L3P,1675.0,1.0,1.0,549.0
1,L3R,2000.0,1.0,1.0,750.0
2,L3T,1990.0,1.0,1.0,600.0


In [71]:
# match postal codes with the two datasets
merged_data = geo_coords.join(zip_group.set_index("postal_code"), on = "postal_code", how = "inner")

In [73]:
merged_data.reset_index()

,index,postal_code,latitude,longitude,rent,num_bedrooms,num_bathrooms,num_sqft
0,4,M1H,43.773136,-79.239476,1902.272727,1.0,1.000000,618.727273
1,9,M1N,43.692657,-79.264848,2050.000000,1.0,1.000000,735.000000
2,10,M1P,43.757410,-79.273304,1936.580645,1.0,0.983871,624.129032
3,12,M1S,43.794200,-79.262029,1852.500000,1.0,1.100000,592.000000
4,13,M1T,43.781638,-79.304302,1950.000000,1.0,1.000000,649.000000
5,14,M1V,43.815252,-79.284577,1795.000000,1.0,1.000000,725.000000
6,18,M2J,43.778517,-79.346556,1966.000000,1.0,1.000000,573.000000
7,19,M2K,43.786947,-79.385975,2048.076923,1.0,1.173077,629.423077
8,21,M2M,43.789053,-79.408493,1975.000000,1.0,1.000000,675.000000
9,22,M2N,43.770120,-79.408493,2061.250000,1.0,1.000000,590.500000


In [ ]:
geo_coords.dtypes

In [ ]:
geo_coords.head()